In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_info = {
    "db_username" : 'user',
    "db_password" : 'aaaa',
    "db_host" : 'localhost',
    "db_port" : '5432',
    "db_name" : 'airline_sentiment'
}

In [3]:
engine = create_engine(f'postgresql://{db_info["db_username"]}:{db_info["db_password"]}@{db_info["db_host"]}:{db_info["db_port"]}/{db_info["db_name"]}')

## Extract data from the raw table

In [4]:
query = f"SELECT * FROM raw_airline_sentiment"

In [5]:
df = pd.read_sql(query,engine)

In [6]:
print("Data extracted successfully. Shape:", df.shape)
df.head()

Data extracted successfully. Shape: (14640, 20)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2/24/15 11:35,5.703060e+17,None,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2/24/15 11:15,5.703010e+17,None,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2/24/15 11:15,5.703010e+17,None,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2/24/15 11:14,5.703010e+17,None,Pacific Time (US & Canada)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   _unit_id                      14640 non-null  int64  
 1   _golden                       14640 non-null  bool   
 2   _unit_state                   14640 non-null  object 
 3   _trusted_judgments            14640 non-null  int64  
 4   _last_judgment_at             14584 non-null  object 
 5   airline_sentiment             14640 non-null  object 
 6   airline_sentiment:confidence  14640 non-null  float64
 7   negativereason                9178 non-null   object 
 8   negativereason:confidence     10522 non-null  float64
 9   airline                       14640 non-null  object 
 10  airline_sentiment_gold        40 non-null     object 
 11  name                          14640 non-null  object 
 12  negativereason_gold           32 non-null     object 
 13  r

In [8]:
# Delete the index column
#df.reset_index(drop = True)

##  Transformation Process

In [9]:
# Convert tweet_created to datetime
df['tweet_created'] = pd.to_datetime(df['tweet_created'])
df['tweet_created'] = pd.to_datetime(df['tweet_created'], format="%Y-%m-%d %H:%M:%S")

C:\Users\ibrahim\AppData\Local\Temp\ipykernel_5348\672895531.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['tweet_created'] = pd.to_datetime(df['tweet_created'])


In [10]:
df["tweet_created"].head(2)

0   2015-02-24 11:35:00
1   2015-02-24 11:15:00
Name: tweet_created, dtype: datetime64[ns]

In [11]:
df['tweet_date'] = df['tweet_created'].dt.date
df['tweet_time'] = df['tweet_created'].dt.time

In [12]:
import nltk
from nltk.corpus import stopwords
import re

In [13]:
nltk.download('stopwords', quiet=True)

True

In [14]:
df["text"].head(10)

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
5    @VirginAmerica seriously would pay $30 a fligh...
6    @VirginAmerica yes, nearly every time I fly VX...
7    @VirginAmerica Really missed a prime opportuni...
8    @virginamerica Well, I didn'tÛ_but NOW I DO! :-D
9    @VirginAmerica it was amazing, and arrived an ...
Name: text, dtype: object

In [15]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text
df['cleaned_text'] = df['text'].apply(clean_text)

In [16]:
df["cleaned_text"].head(10)

0                                                 said
1        plus youve added commercials experience tacky
2         didnt today must mean need take another trip
3    really aggressive blast obnoxious entertainmen...
4                                 really big bad thing
5    seriously would pay flight seats didnt playing...
6    yes nearly every time fly vx ear worm wont go ...
7    really missed prime opportunity men without ha...
8                                        well didntbut
9                amazing arrived hour early youre good
Name: cleaned_text, dtype: object

In [17]:
# Convert sentiment to numerical values
df["sentiment_score"] = df["airline_sentiment"].map({ "negative": -1,"positive":1, "neutral":0 })

In [18]:
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,...,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone,tweet_date,tweet_time,cleaned_text,sentiment_score
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,None,NaN,Virgin America,...,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:00,5.703060e+17,None,Eastern Time (US & Canada),2015-02-24,11:35:00,said,0
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,None,0.0000,Virgin America,...,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:00,5.703010e+17,None,Pacific Time (US & Canada),2015-02-24,11:15:00,plus youve added commercials experience tacky,1
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,None,NaN,Virgin America,...,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:00,5.703010e+17,Lets Play,Central Time (US & Canada),2015-02-24,11:15:00,didnt today must mean need take another trip,0
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,...,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:00,5.703010e+17,None,Pacific Time (US & Canada),2015-02-24,11:15:00,really aggressive blast obnoxious entertainmen...,-1
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,...,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:00,5.703010e+17,None,Pacific Time (US & Canada),2015-02-24,11:14:00,really big bad thing,-1


In [19]:
columns_to_keep = ['airline', 'airline_sentiment', 'sentiment_score', 'negativereason', 
                   'text', 'cleaned_text', 'tweet_created', 'tweet_date', 'tweet_time', 
                   'retweet_count', 'tweet_location']

In [20]:
df_transformed = df[columns_to_keep]

In [22]:
df_transformed.shape

(14640, 11)

In [25]:
df_transformed.to_csv("../data/processed/twitter_airlines_sentiment_processed.csv",index = False)